In [1]:
import os, shutil, random
import numpy as np
import cv2
import random
from tqdm import tqdm_notebook
import shutil

## Clear_boundary

In [3]:
imgs_dir = r"~\data\imgs_feature_extraction_deep_learning\imgs_boundary_clear\train"
n_labels = len(os.listdir(imgs_dir))

label_dir = os.path.split(imgs_dir)[0]+'\\labels'
if not os.path.exists(label_dir):
    os.makedirs(label_dir)
    
imgs_0_dir = os.path.join(imgs_dir, '0')
imgs_1_dir = os.path.join(imgs_dir, '1')

file_0_all = os.listdir(imgs_0_dir)
file_1_all = os.listdir(imgs_1_dir)

random.shuffle(file_1_all)
random.shuffle(file_0_all)

if n_labels == 3:
    imgs_2_dir = os.path.join(imgs_dir, '2')
    file_2_all = os.listdir(imgs_2_dir)
    random.shuffle(file_2_all)
    
print(len(file_0_all),len(file_1_all),len(file_2_all)) if n_labels == 3  else print(len(file_0_all),len(file_1_all))

for i in range(1, 11):
    rate = i * 0.1
    file_0_list = file_0_all[:int(len(file_0_all)*rate)]
    file_1_list = file_1_all[:int(len(file_1_all)*rate)]

    for j in range(len(file_0_list)):
        file_0_list[j] = imgs_dir + '\\0\\' + file_0_list[j] + ',0'
    for k in range(len(file_1_list)):
        file_1_list[k] = imgs_dir + '\\1\\' + file_1_list[k] + ',1'
    
    label_list = file_0_list[:int(len(file_0_list)*0.9)] + file_1_list[:int(len(file_1_list)*0.9)]
    val_list = file_0_list[int(len(file_0_list)*0.9):] + file_1_list[int(len(file_1_list)*0.9):]
    
    if n_labels == 3:
        file_2_list = file_2_all[:int(len(file_2_all)*rate)]
        
        for q in range(len(file_2_list)):
            file_2_list[q] = imgs_dir + '\\2\\' + file_2_list[q] + ',2'
            
        label_list += file_2_list[:int(len(file_2_list)*0.9)]
        val_list += file_2_list[int(len(file_2_list)*0.9):]
    
    if n_labels == 3:
        print(len(label_list), len(val_list), len(file_0_list), len(file_1_list), len(file_2_list)) 
    else:
        print(len(label_list), len(val_list), len(file_0_list), len(file_1_list))
    
    
    label_txt_path = os.path.join(label_dir, 'label_'+str(i*10)+'.txt')
    val_label_txt_path = os.path.join(label_dir, 'val_label_'+str(i*10)+'.txt')

    with open(label_txt_path, 'w', encoding="utf-8") as f:
        for l in range(len(label_list)):
            f.write(label_list[l]+'\n')
        f.close()
        
    with open(val_label_txt_path, 'w', encoding="utf-8") as f:
        for m in range(len(val_list)):
            f.write(val_list[m]+'\n')
        f.close()
        
for i in range(1, 11):
    train_list = label_list+val_list

    val_label_txt_path = os.path.join(label_dir, 'val_label_'+str(i*10)+'.txt')
    train_txt_path = os.path.join(label_dir, 'train_'+str(i*10)+'.txt')
    
    f_val = open(val_label_txt_path, 'r', encoding='utf-8')
    val_file_list = f_val.readlines()
    for val_file in val_file_list:
        if val_file[:-1] in train_list:

            train_list.remove(val_file[:-1])

    with open(train_txt_path, 'w', encoding="utf-8") as f_train:
        for n in range(len(train_list)):
            f_train.write(train_list[n]+'\n')
        f_train.close()
print('finished')

In [5]:
# randomly select 400 images from each group, calculate the mean and std

root = os.path.split(os.path.split(imgs_dir)[0])[0]
train_root = root + r'\\imgs_boundary_clear\train'
val_path = root + r'\imgs_boundary_clear'
subs = os.listdir(train_root)
min_imgs = 10000000
subs = os.listdir(train_root)
for sub in subs:
    abs_sub = os.path.join(train_root, sub)
    files = os.listdir(abs_sub)
    count = len(files)
    print(sub, ':', count)
    if count < min_imgs:
        min_imgs = count       
img_count = min(400, min_imgs)
print('to chk :', img_count)

img_h,img_w = 224,224
imgs = np.zeros([img_w,img_h,3,1])
means,stdevs = [],[]

subs = os.listdir(train_root)
for sub in subs:
    abs_sub = os.path.join(train_root, sub)
    files = os.listdir(abs_sub)
    random.shuffle(files)
    
    for i in tqdm_notebook(range(img_count)):
        img = cv2.imdecode(np.fromfile(os.path.join(abs_sub, files[i]), dtype=np.uint8), -1)
        img = cv2.resize(img,(img_h,img_w))
        img = img[:,:,:,np.newaxis]

        imgs = np.concatenate((imgs,img),axis=3)
              
imgs = imgs[:,:,:,1:]
print(imgs.shape)
imgs = imgs.astype(np.float32)/255.
 
for i in tqdm_notebook(range(3)):
    pixels = imgs[:,i,:].ravel() # pull in a row
    means.append(round(np.mean(pixels), 4))
    stdevs.append(round(np.std(pixels), 4))
 
# The image format read by cv2 is BGR, and the image read by PIL/Skimage is RGB without need for conversion
means.reverse() # BGR --> RGB
stdevs.reverse()
 
print("normMean = {}".format(means))
print("normStd = {}".format(stdevs))
print('transforms.Normalize(normMean = {},normStd = {})'.format(means,stdevs))

## Surface_rough

In [6]:
imgs_dir = r"~\data\imgs_feature_extraction_deep_learning\imgs_surface_rough\train"
n_labels = len(os.listdir(imgs_dir))


label_dir = os.path.split(imgs_dir)[0]+'\\labels'
if not os.path.exists(label_dir):
    os.makedirs(label_dir)


imgs_0_dir = os.path.join(imgs_dir, '0')
imgs_1_dir = os.path.join(imgs_dir, '1')

file_0_all = os.listdir(imgs_0_dir)
file_1_all = os.listdir(imgs_1_dir)

random.shuffle(file_1_all)
random.shuffle(file_0_all)

if n_labels == 3:
    imgs_2_dir = os.path.join(imgs_dir, '2')
    file_2_all = os.listdir(imgs_2_dir)
    random.shuffle(file_2_all)
    

print(len(file_0_all),len(file_1_all),len(file_2_all)) if n_labels == 3  else print(len(file_0_all),len(file_1_all))

for i in range(1, 11):
    rate = i * 0.1
    file_0_list = file_0_all[:int(len(file_0_all)*rate)]
    file_1_list = file_1_all[:int(len(file_1_all)*rate)]

    for j in range(len(file_0_list)):
        file_0_list[j] = imgs_dir + '\\0\\' + file_0_list[j] + ',0'
    for k in range(len(file_1_list)):
        file_1_list[k] = imgs_dir + '\\1\\' + file_1_list[k] + ',1'
    
    label_list = file_0_list[:int(len(file_0_list)*0.9)] + file_1_list[:int(len(file_1_list)*0.9)]
    val_list = file_0_list[int(len(file_0_list)*0.9):] + file_1_list[int(len(file_1_list)*0.9):]
    
    if n_labels == 3:
        file_2_list = file_2_all[:int(len(file_2_all)*rate)]
        
        for q in range(len(file_2_list)):
            file_2_list[q] = imgs_dir + '\\2\\' + file_2_list[q] + ',2'
            
        label_list += file_2_list[:int(len(file_2_list)*0.9)]
        val_list += file_2_list[int(len(file_2_list)*0.9):]
    
    if n_labels == 3:
        print(len(label_list), len(val_list), len(file_0_list), len(file_1_list), len(file_2_list)) 
    else:
        print(len(label_list), len(val_list), len(file_0_list), len(file_1_list))
    
    label_txt_path = os.path.join(label_dir, 'label_'+str(i*10)+'.txt')
    val_label_txt_path = os.path.join(label_dir, 'val_label_'+str(i*10)+'.txt')

    with open(label_txt_path, 'w', encoding="utf-8") as f:
        for l in range(len(label_list)):
            f.write(label_list[l]+'\n')
        f.close()
        
    with open(val_label_txt_path, 'w', encoding="utf-8") as f:
        for m in range(len(val_list)):
            f.write(val_list[m]+'\n')
        f.close()
        
for i in range(1, 11):
    train_list = label_list+val_list

    val_label_txt_path = os.path.join(label_dir, 'val_label_'+str(i*10)+'.txt')
    train_txt_path = os.path.join(label_dir, 'train_'+str(i*10)+'.txt')
    
    f_val = open(val_label_txt_path, 'r', encoding='utf-8')
    val_file_list = f_val.readlines()
    for val_file in val_file_list:
        if val_file[:-1] in train_list:

            train_list.remove(val_file[:-1])

    with open(train_txt_path, 'w', encoding="utf-8") as f_train:
        for n in range(len(train_list)):
            f_train.write(train_list[n]+'\n')
        f_train.close()
print('finished')

In [7]:
# randomly select 400 images from each group, calculate the mean and std

root = os.path.split(os.path.split(imgs_dir)[0])[0]
train_root = root + r'\imgs_surface_rough\train'
val_path = root + r'\imgs_surface_rough'
subs = os.listdir(train_root)
min_imgs = 10000000
subs = os.listdir(train_root)
for sub in subs:
    abs_sub = os.path.join(train_root, sub)
    files = os.listdir(abs_sub)
    count = len(files)
    print(sub, ':', count)
    if count < min_imgs:
        min_imgs = count       
img_count = min(400, min_imgs)
print('to chk :', img_count)
img_h,img_w = 224,224
imgs = np.zeros([img_w,img_h,3,1])
means,stdevs = [],[]

subs = os.listdir(train_root)
for sub in subs:
    abs_sub = os.path.join(train_root, sub)
    files = os.listdir(abs_sub)
    random.shuffle(files)
    
    for i in tqdm_notebook(range(img_count)):
        img = cv2.imdecode(np.fromfile(os.path.join(abs_sub, files[i]), dtype=np.uint8), -1)
        img = cv2.resize(img,(img_h,img_w))
        img = img[:,:,:,np.newaxis]

        imgs = np.concatenate((imgs,img),axis=3)
        
        
imgs = imgs[:,:,:,1:]
print(imgs.shape)
imgs = imgs.astype(np.float32)/255.
 
for i in tqdm_notebook(range(3)):
    pixels = imgs[:,i,:].ravel() # pull in a row
    means.append(round(np.mean(pixels), 4))
    stdevs.append(round(np.std(pixels), 4))
 
means.reverse() # BGR --> RGB
stdevs.reverse()
 
print("normMean = {}".format(means))
print("normStd = {}".format(stdevs))
print('transforms.Normalize(normMean = {},normStd = {})'.format(means,stdevs))

## Bleeding

In [8]:
imgs_dir = r"~\data\imgs_feature_extraction_deep_learning\imgs_bleeding\train"
n_labels = len(os.listdir(imgs_dir))

label_dir = os.path.split(imgs_dir)[0]+'\\labels'
if not os.path.exists(label_dir):
    os.makedirs(label_dir)

imgs_0_dir = os.path.join(imgs_dir, '0')
imgs_1_dir = os.path.join(imgs_dir, '1')

file_0_all = os.listdir(imgs_0_dir)
file_1_all = os.listdir(imgs_1_dir)

random.shuffle(file_1_all)
random.shuffle(file_0_all)


if n_labels == 3:
    imgs_2_dir = os.path.join(imgs_dir, '2')
    file_2_all = os.listdir(imgs_2_dir)
    random.shuffle(file_2_all)
    

print(len(file_0_all),len(file_1_all),len(file_2_all)) if n_labels == 3  else print(len(file_0_all),len(file_1_all))

for i in range(1, 11):
    rate = i * 0.1
    file_0_list = file_0_all[:int(len(file_0_all)*rate)]
    file_1_list = file_1_all[:int(len(file_1_all)*rate)]

    for j in range(len(file_0_list)):
        file_0_list[j] = imgs_dir + '\\0\\' + file_0_list[j] + ',0'
    for k in range(len(file_1_list)):
        file_1_list[k] = imgs_dir + '\\1\\' + file_1_list[k] + ',1'
    
    label_list = file_0_list[:int(len(file_0_list)*0.9)] + file_1_list[:int(len(file_1_list)*0.9)]
    val_list = file_0_list[int(len(file_0_list)*0.9):] + file_1_list[int(len(file_1_list)*0.9):]
    
    if n_labels == 3:
        file_2_list = file_2_all[:int(len(file_2_all)*rate)]
        
        for q in range(len(file_2_list)):
            file_2_list[q] = imgs_dir + '\\2\\' + file_2_list[q] + ',2'
            
        label_list += file_2_list[:int(len(file_2_list)*0.9)]
        val_list += file_2_list[int(len(file_2_list)*0.9):]
    
    if n_labels == 3:
        print(len(label_list), len(val_list), len(file_0_list), len(file_1_list), len(file_2_list)) 
    else:
        print(len(label_list), len(val_list), len(file_0_list), len(file_1_list))
    
    
    label_txt_path = os.path.join(label_dir, 'label_'+str(i*10)+'.txt')
    val_label_txt_path = os.path.join(label_dir, 'val_label_'+str(i*10)+'.txt')

    with open(label_txt_path, 'w', encoding="utf-8") as f:
        for l in range(len(label_list)):
            f.write(label_list[l]+'\n')
        f.close()
        
    with open(val_label_txt_path, 'w', encoding="utf-8") as f:
        for m in range(len(val_list)):
            f.write(val_list[m]+'\n')
        f.close()
        
for i in range(1, 11):
    train_list = label_list+val_list

    val_label_txt_path = os.path.join(label_dir, 'val_label_'+str(i*10)+'.txt')
    train_txt_path = os.path.join(label_dir, 'train_'+str(i*10)+'.txt')
    
    f_val = open(val_label_txt_path, 'r', encoding='utf-8')
    val_file_list = f_val.readlines()
    for val_file in val_file_list:
        if val_file[:-1] in train_list:

            train_list.remove(val_file[:-1])

    with open(train_txt_path, 'w', encoding="utf-8") as f_train:
        for n in range(len(train_list)):
            f_train.write(train_list[n]+'\n')
        f_train.close()
print('finished')

In [9]:
# randomly select 400 images from each group, calculate the mean and std

root = os.path.split(os.path.split(imgs_dir)[0])[0]
train_root = root + r'\imgs_bleeding\train'
val_path = root + r'\imgs_bleeding'
subs = os.listdir(train_root)
min_imgs = 10000000
subs = os.listdir(train_root)
for sub in subs:
    abs_sub = os.path.join(train_root, sub)
    files = os.listdir(abs_sub)
    count = len(files)
    print(sub, ':', count)
    if count < min_imgs:
        min_imgs = count       
img_count = min(400, min_imgs)
print('to chk :', img_count)
img_h,img_w = 224,224
imgs = np.zeros([img_w,img_h,3,1])
means,stdevs = [],[]

subs = os.listdir(train_root)
for sub in subs:
    abs_sub = os.path.join(train_root, sub)
    files = os.listdir(abs_sub)
    random.shuffle(files)
    
    for i in tqdm_notebook(range(img_count)):
        img = cv2.imdecode(np.fromfile(os.path.join(abs_sub, files[i]), dtype=np.uint8), -1)
        img = cv2.resize(img,(img_h,img_w))
        img = img[:,:,:,np.newaxis]

        imgs = np.concatenate((imgs,img),axis=3)
        
        
imgs = imgs[:,:,:,1:]
print(imgs.shape)
imgs = imgs.astype(np.float32)/255.
 
for i in tqdm_notebook(range(3)):
    pixels = imgs[:,i,:].ravel() # pull in a row
    means.append(round(np.mean(pixels), 4))
    stdevs.append(round(np.std(pixels), 4))
 
means.reverse() # BGR --> RGB
stdevs.reverse()
 
print("normMean = {}".format(means))
print("normStd = {}".format(stdevs))
print('transforms.Normalize(normMean = {},normStd = {})'.format(means,stdevs))

## Tone

In [10]:
imgs_dir = r"~\data\imgs_feature_extraction_deep_learning\imgs_tone\train"
n_labels = len(os.listdir(imgs_dir))

label_dir = os.path.split(imgs_dir)[0]+'\\labels'
if not os.path.exists(label_dir):
    os.makedirs(label_dir)

imgs_0_dir = os.path.join(imgs_dir, '0')
imgs_1_dir = os.path.join(imgs_dir, '1')

file_0_all = os.listdir(imgs_0_dir)
file_1_all = os.listdir(imgs_1_dir)

random.shuffle(file_1_all)
random.shuffle(file_0_all)


if n_labels == 3:
    imgs_2_dir = os.path.join(imgs_dir, '2')
    file_2_all = os.listdir(imgs_2_dir)
    random.shuffle(file_2_all)
    

print(len(file_0_all),len(file_1_all),len(file_2_all)) if n_labels == 3  else print(len(file_0_all),len(file_1_all))

for i in range(1, 11):
    rate = i * 0.1
    file_0_list = file_0_all[:int(len(file_0_all)*rate)]
    file_1_list = file_1_all[:int(len(file_1_all)*rate)]

    for j in range(len(file_0_list)):
        file_0_list[j] = imgs_dir + '\\0\\' + file_0_list[j] + ',0'
    for k in range(len(file_1_list)):
        file_1_list[k] = imgs_dir + '\\1\\' + file_1_list[k] + ',1'
    
    label_list = file_0_list[:int(len(file_0_list)*0.9)] + file_1_list[:int(len(file_1_list)*0.9)]
    val_list = file_0_list[int(len(file_0_list)*0.9):] + file_1_list[int(len(file_1_list)*0.9):]
    
    if n_labels == 3:
        file_2_list = file_2_all[:int(len(file_2_all)*rate)]
        
        for q in range(len(file_2_list)):
            file_2_list[q] = imgs_dir + '\\2\\' + file_2_list[q] + ',2'
            
        label_list += file_2_list[:int(len(file_2_list)*0.9)]
        val_list += file_2_list[int(len(file_2_list)*0.9):]
    
    if n_labels == 3:
        print(len(label_list), len(val_list), len(file_0_list), len(file_1_list), len(file_2_list)) 
    else:
        print(len(label_list), len(val_list), len(file_0_list), len(file_1_list))
    

    label_txt_path = os.path.join(label_dir, 'label_'+str(i*10)+'.txt')
    val_label_txt_path = os.path.join(label_dir, 'val_label_'+str(i*10)+'.txt')


    with open(label_txt_path, 'w', encoding="utf-8") as f:
        for l in range(len(label_list)):
            f.write(label_list[l]+'\n')
        f.close()
        
    with open(val_label_txt_path, 'w', encoding="utf-8") as f:
        for m in range(len(val_list)):
            f.write(val_list[m]+'\n')
        f.close()
        
for i in range(1, 11):
    train_list = label_list+val_list

    val_label_txt_path = os.path.join(label_dir, 'val_label_'+str(i*10)+'.txt')
    train_txt_path = os.path.join(label_dir, 'train_'+str(i*10)+'.txt')
    
    f_val = open(val_label_txt_path, 'r', encoding='utf-8')
    val_file_list = f_val.readlines()
    for val_file in val_file_list:
        if val_file[:-1] in train_list:

            train_list.remove(val_file[:-1])

    with open(train_txt_path, 'w', encoding="utf-8") as f_train:
        for n in range(len(train_list)):
            f_train.write(train_list[n]+'\n')
        f_train.close()
print('finished')

In [11]:
# randomly select 400 images from each group, calculate the mean and std

root = os.path.split(os.path.split(imgs_dir)[0])[0]
train_root = root + r'\imgs_tone\train'
val_path = root + r'\imgs_tone'
subs = os.listdir(train_root)
min_imgs = 10000000
subs = os.listdir(train_root)
for sub in subs:
    abs_sub = os.path.join(train_root, sub)
    files = os.listdir(abs_sub)
    count = len(files)
    print(sub, ':', count)
    if count < min_imgs:
        min_imgs = count       
img_count = min(400, min_imgs)
print('to chk :', img_count)
img_h,img_w = 224,224
imgs = np.zeros([img_w,img_h,3,1])
means,stdevs = [],[]

subs = os.listdir(train_root)
for sub in subs:
    abs_sub = os.path.join(train_root, sub)
    files = os.listdir(abs_sub)
    random.shuffle(files)
    
    for i in tqdm_notebook(range(img_count)):
        img = cv2.imdecode(np.fromfile(os.path.join(abs_sub, files[i]), dtype=np.uint8), -1)
        img = cv2.resize(img,(img_h,img_w))
        img = img[:,:,:,np.newaxis]

        imgs = np.concatenate((imgs,img),axis=3)
        
        
imgs = imgs[:,:,:,1:]
print(imgs.shape)
imgs = imgs.astype(np.float32)/255.
 
for i in tqdm_notebook(range(3)):
    pixels = imgs[:,i,:].ravel() # pull in a row
    means.append(round(np.mean(pixels), 4))
    stdevs.append(round(np.std(pixels), 4))
 
means.reverse() # BGR --> RGB
stdevs.reverse()
 
print("normMean = {}".format(means))
print("normStd = {}".format(stdevs))
print('transforms.Normalize(normMean = {},normStd = {})'.format(means,stdevs))

## Elevated

In [12]:
imgs_dir = r"~\data\imgs_feature_extraction_deep_learning\imgs_elevated\train"
n_labels = len(os.listdir(imgs_dir))


label_dir = os.path.split(imgs_dir)[0]+'\\labels'
if not os.path.exists(label_dir):
    os.makedirs(label_dir)


imgs_0_dir = os.path.join(imgs_dir, '0')
imgs_1_dir = os.path.join(imgs_dir, '1')

file_0_all = os.listdir(imgs_0_dir)
file_1_all = os.listdir(imgs_1_dir)

random.shuffle(file_1_all)
random.shuffle(file_0_all)


if n_labels == 3:
    imgs_2_dir = os.path.join(imgs_dir, '2')
    file_2_all = os.listdir(imgs_2_dir)
    random.shuffle(file_2_all)
    

print(len(file_0_all),len(file_1_all),len(file_2_all)) if n_labels == 3  else print(len(file_0_all),len(file_1_all))

for i in range(1, 11):
    rate = i * 0.1
    file_0_list = file_0_all[:int(len(file_0_all)*rate)]
    file_1_list = file_1_all[:int(len(file_1_all)*rate)]

    for j in range(len(file_0_list)):
        file_0_list[j] = imgs_dir + '\\0\\' + file_0_list[j] + ',0'
    for k in range(len(file_1_list)):
        file_1_list[k] = imgs_dir + '\\1\\' + file_1_list[k] + ',1'
    
    label_list = file_0_list[:int(len(file_0_list)*0.9)] + file_1_list[:int(len(file_1_list)*0.9)]
    val_list = file_0_list[int(len(file_0_list)*0.9):] + file_1_list[int(len(file_1_list)*0.9):]
    
    if n_labels == 3:
        file_2_list = file_2_all[:int(len(file_2_all)*rate)]
        
        for q in range(len(file_2_list)):
            file_2_list[q] = imgs_dir + '\\2\\' + file_2_list[q] + ',2'
            
        label_list += file_2_list[:int(len(file_2_list)*0.9)]
        val_list += file_2_list[int(len(file_2_list)*0.9):]
    
    if n_labels == 3:
        print(len(label_list), len(val_list), len(file_0_list), len(file_1_list), len(file_2_list)) 
    else:
        print(len(label_list), len(val_list), len(file_0_list), len(file_1_list))
    

    label_txt_path = os.path.join(label_dir, 'label_'+str(i*10)+'.txt')
    val_label_txt_path = os.path.join(label_dir, 'val_label_'+str(i*10)+'.txt')


    with open(label_txt_path, 'w', encoding="utf-8") as f:
        for l in range(len(label_list)):
            f.write(label_list[l]+'\n')
        f.close()
        

    with open(val_label_txt_path, 'w', encoding="utf-8") as f:
        for m in range(len(val_list)):
            f.write(val_list[m]+'\n')
        f.close()
        

for i in range(1, 11):
    train_list = label_list+val_list

    val_label_txt_path = os.path.join(label_dir, 'val_label_'+str(i*10)+'.txt')
    train_txt_path = os.path.join(label_dir, 'train_'+str(i*10)+'.txt')
    
    f_val = open(val_label_txt_path, 'r', encoding='utf-8')
    val_file_list = f_val.readlines()
    for val_file in val_file_list:
        if val_file[:-1] in train_list:

            train_list.remove(val_file[:-1])

    with open(train_txt_path, 'w', encoding="utf-8") as f_train:
        for n in range(len(train_list)):
            f_train.write(train_list[n]+'\n')
        f_train.close()
print('finished')

In [13]:
# randomly select 400 images from each group, calculate the mean and std

root = os.path.split(os.path.split(imgs_dir)[0])[0]
train_root = root + r'\imgs_elevated\train'
val_path = root + r'\imgs_elevated'
subs = os.listdir(train_root)
min_imgs = 10000000
subs = os.listdir(train_root)
for sub in subs:
    abs_sub = os.path.join(train_root, sub)
    files = os.listdir(abs_sub)
    count = len(files)
    print(sub, ':', count)
    if count < min_imgs:
        min_imgs = count       
img_count = min(400, min_imgs)
print('to chk :', img_count)
img_h,img_w = 224,224
imgs = np.zeros([img_w,img_h,3,1])
means,stdevs = [],[]

subs = os.listdir(train_root)
for sub in subs:
    abs_sub = os.path.join(train_root, sub)
    files = os.listdir(abs_sub)
    random.shuffle(files)
    
    for i in tqdm_notebook(range(img_count)):
        img = cv2.imdecode(np.fromfile(os.path.join(abs_sub, files[i]), dtype=np.uint8), -1)
        img = cv2.resize(img,(img_h,img_w))
        img = img[:,:,:,np.newaxis]

        imgs = np.concatenate((imgs,img),axis=3)
        
        
imgs = imgs[:,:,:,1:]
print(imgs.shape)
imgs = imgs.astype(np.float32)/255.
 
for i in tqdm_notebook(range(3)):
    pixels = imgs[:,i,:].ravel() # pull in a row
    means.append(round(np.mean(pixels), 4))
    stdevs.append(round(np.std(pixels), 4))
 
means.reverse() # BGR --> RGB
stdevs.reverse()
 
print("normMean = {}".format(means))
print("normStd = {}".format(stdevs))
print('transforms.Normalize(normMean = {},normStd = {})'.format(means,stdevs))

### Depressed

In [14]:
imgs_dir = r"~\data\imgs_feature_extraction_deep_learning\imgs_depressed\train"
n_labels = len(os.listdir(imgs_dir))


label_dir = os.path.split(imgs_dir)[0]+'\\labels'
if not os.path.exists(label_dir):
    os.makedirs(label_dir)


imgs_0_dir = os.path.join(imgs_dir, '0')
imgs_1_dir = os.path.join(imgs_dir, '1')

file_0_all = os.listdir(imgs_0_dir)
file_1_all = os.listdir(imgs_1_dir)

random.shuffle(file_1_all)
random.shuffle(file_0_all)


if n_labels == 3:
    imgs_2_dir = os.path.join(imgs_dir, '2')
    file_2_all = os.listdir(imgs_2_dir)
    random.shuffle(file_2_all)
    

print(len(file_0_all),len(file_1_all),len(file_2_all)) if n_labels == 3  else print(len(file_0_all),len(file_1_all))

for i in range(1, 11):
    rate = i * 0.1
    file_0_list = file_0_all[:int(len(file_0_all)*rate)]
    file_1_list = file_1_all[:int(len(file_1_all)*rate)]

    for j in range(len(file_0_list)):
        file_0_list[j] = imgs_dir + '\\0\\' + file_0_list[j] + ',0'
    for k in range(len(file_1_list)):
        file_1_list[k] = imgs_dir + '\\1\\' + file_1_list[k] + ',1'
    
    label_list = file_0_list[:int(len(file_0_list)*0.9)] + file_1_list[:int(len(file_1_list)*0.9)]
    val_list = file_0_list[int(len(file_0_list)*0.9):] + file_1_list[int(len(file_1_list)*0.9):]
    
    if n_labels == 3:
        file_2_list = file_2_all[:int(len(file_2_all)*rate)]
        
        for q in range(len(file_2_list)):
            file_2_list[q] = imgs_dir + '\\2\\' + file_2_list[q] + ',2'
            
        label_list += file_2_list[:int(len(file_2_list)*0.9)]
        val_list += file_2_list[int(len(file_2_list)*0.9):]
    
    if n_labels == 3:
        print(len(label_list), len(val_list), len(file_0_list), len(file_1_list), len(file_2_list)) 
    else:
        print(len(label_list), len(val_list), len(file_0_list), len(file_1_list))
    

    label_txt_path = os.path.join(label_dir, 'label_'+str(i*10)+'.txt')
    val_label_txt_path = os.path.join(label_dir, 'val_label_'+str(i*10)+'.txt')

    with open(label_txt_path, 'w', encoding="utf-8") as f:
        for l in range(len(label_list)):
            f.write(label_list[l]+'\n')
        f.close()
        
    with open(val_label_txt_path, 'w', encoding="utf-8") as f:
        for m in range(len(val_list)):
            f.write(val_list[m]+'\n')
        f.close()
        

for i in range(1, 11):
    train_list = label_list+val_list

    val_label_txt_path = os.path.join(label_dir, 'val_label_'+str(i*10)+'.txt')
    train_txt_path = os.path.join(label_dir, 'train_'+str(i*10)+'.txt')
    
    f_val = open(val_label_txt_path, 'r', encoding='utf-8')
    val_file_list = f_val.readlines()
    for val_file in val_file_list:
        if val_file[:-1] in train_list:

            train_list.remove(val_file[:-1])

    with open(train_txt_path, 'w', encoding="utf-8") as f_train:
        for n in range(len(train_list)):
            f_train.write(train_list[n]+'\n')
        f_train.close()
print('finished')

In [15]:
# randomly select 400 images from each group, calculate the mean and std

root = os.path.split(os.path.split(imgs_dir)[0])[0]
train_root = root + r'\imgs_depressed\train'
val_path = root + r'\imgs_depressed'
subs = os.listdir(train_root)
min_imgs = 10000000
subs = os.listdir(train_root)
for sub in subs:
    abs_sub = os.path.join(train_root, sub)
    files = os.listdir(abs_sub)
    count = len(files)
    print(sub, ':', count)
    if count < min_imgs:
        min_imgs = count       
img_count = min(400, min_imgs)
print('to chk :', img_count)
img_h,img_w = 224,224
imgs = np.zeros([img_w,img_h,3,1])
means,stdevs = [],[]

subs = os.listdir(train_root)
for sub in subs:
    abs_sub = os.path.join(train_root, sub)
    files = os.listdir(abs_sub)
    random.shuffle(files)
    
    for i in tqdm_notebook(range(img_count)):
        img = cv2.imdecode(np.fromfile(os.path.join(abs_sub, files[i]), dtype=np.uint8), -1)
        img = cv2.resize(img,(img_h,img_w))
        img = img[:,:,:,np.newaxis]

        imgs = np.concatenate((imgs,img),axis=3)
        
        
imgs = imgs[:,:,:,1:]
print(imgs.shape)
imgs = imgs.astype(np.float32)/255.
 
for i in tqdm_notebook(range(3)):
    pixels = imgs[:,i,:].ravel() # pull in a row
    means.append(round(np.mean(pixels), 4))
    stdevs.append(round(np.std(pixels), 4))
 
means.reverse() # BGR --> RGB
stdevs.reverse()
 
print("normMean = {}".format(means))
print("normStd = {}".format(stdevs))
print('transforms.Normalize(normMean = {},normStd = {})'.format(means,stdevs))